# Image Classifier Evaluation
## Focus & Astigmatism Performance Analysis

- **Author:** [Aaron Woods](https://aaronwoods.info)
- **Date Created:** October 26, 2023
- **Objective:** This notebook is designated for the evaluation phase, concentrating on in-depth analysis of the model's performance and application in real-world scenarios.

## Evaluation Agenda
1. **Model Loading**: Retrieve the trained model for analysis.
2. **Performance Metrics Analysis**: Examine key performance indicators and metrics.
3. **Real-world Testing**: Validate model efficacy in practical scenarios.

## Further Actions
- Based on the outcomes and insights obtained here, consider revisiting the [Training Notebook](AI_Classify_Train_v1.ipynb) to refine or retrain the model, if necessary.


In [ ]:
# Setup Instructions
# 1. Run the shared setup script to prepare the environment and integrate dependencies.
# 2. Execute the setup configurations.

# Import and run the setup function from the setup module
from setup import setup
setup()
